In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from IPython.display import clear_output
import seaborn as sns

import cPickle as pk
import pandas as pd
import networkx as nx
import scipy.sparse as sp
import itertools as itls
from itertools import combinations
import numpy as np
from math import sqrt

import sys

# Speed Testing Net Similatiry By DeltaCon

## Importing
[to top](#Table-of-Contents) 

In [2]:
pwd

u'/Users/innovation-sfi/Dropbox/Research/Innovation/EPO/Data_processing/Python'

In [3]:
lsls = pk.load(open('pickles_and_munged_csv/lsls_US_2010.pk', 'rb')); lsls

[[6755, 7302],
 [6284],
 [3310, 3310, 3311],
 [766, 776],
 [7315, 7316, 7316],
 [774, 791],
 [774, 774, 775, 775, 776],
 [7292, 7292, 7351, 7368],
 [3412, 3412, 3412],
 [3452, 3457],
 [766, 766],
 [6337, 6337],
 [7351, 7351],
 [330,
  330,
  330,
  330,
  330,
  854,
  854,
  856,
  856,
  858,
  858,
  861,
  912,
  912,
  912,
  912,
  926,
  929,
  933],
 [2850],
 [321, 321, 321, 321, 332, 332, 332, 333],
 [2749, 2749, 6513, 6523],
 [5585, 5585],
 [5530, 5530, 5531],
 [1142, 5094, 5194, 5214],
 [6225, 6987, 6989],
 [2332, 2332, 2332],
 [7268, 7291, 7292, 7301, 7351, 7353, 7353],
 [2427, 2427, 2427],
 [4787, 4787],
 [3119, 3119, 3821, 3821],
 [1563, 1616],
 [323, 323, 323, 327, 328, 328, 332],
 [7292, 7292],
 [4743, 4751],
 [4743, 4751],
 [7261, 7360, 7360],
 [4838],
 [3716, 3738, 3738],
 [4328],
 [6358, 6358, 6358, 6358],
 [7292, 7292, 7301, 7316, 7323, 7323, 7323],
 [6581, 7352, 7353],
 [6729, 6729, 6729, 6729, 6729],
 [854, 3494, 3494],
 [6966, 6966, 6966, 6966],
 [6625, 6626],
 [

# Speed Testing

## From lsls to Adf
### Fastest found so far -  via NX

In [4]:
num_pats = 10000

In [5]:
len(lsls)

1892590

In [6]:
%%timeit #via nx
g1 = nx.MultiGraph()
for ls in lsls[:num_pats]:
    g1.add_edges_from(combinations(ls, 2))
Adf1 = nx.to_pandas_dataframe(g1)

1 loops, best of 3: 960 ms per loop


In [7]:
%%timeit #via nx
g1 = nx.MultiGraph()
for ls in lsls[:num_pats]:
    g1.add_edges_from(combinations(ls, 2))

1 loops, best of 3: 199 ms per loop


In [9]:
g1 = nx.MultiGraph()
for ls in lsls[:num_pats]:
    g1.add_edges_from(combinations(ls, 2))

In [10]:
%%timeit
Adf1 = nx.to_pandas_dataframe(g1)

1 loops, best of 3: 802 ms per loop


In [ ]:
%%timeit
Adf = pd.DataFrame()
for ls in lsls[:num_pats]:
    for pair in combinations(ls, 2):
        try:
            Adf.loc[pair[0],pair[1]] += 1
            Adf.loc[pair[1],pair[0]] += 1
        except KeyError:
            Adf.loc[pair[0],pair[1]] = 1
            Adf.loc[pair[1],pair[0]] = 1
Adf.fillna(0, inplace=True)

In [ ]:
%%timeit
Adf = pd.DataFrame()
for ls in lsls[:num_pats]:
    for pair in combinations(ls, 2):
        try:
            Adf.loc[pair[0],pair[1]] += 1
            Adf.loc[pair[1],pair[0]] += 1
        except KeyError:
            Adf.loc[pair[0],pair[1]] = 1
            Adf.loc[pair[1],pair[0]] = 1
Adf = Adf.fillna(0)

In [ ]:
%%timeit
Adf = pd.DataFrame()
for ls in lsls[:num_pats]:
    for pair in combinations(ls, 2):
        try:
            Adf.loc[pair[0],pair[1]] += 1
        except KeyError:
            Adf.loc[pair[0],pair[1]] = 1
            
inx = Adf.index
cols = Adf.columns
inx_un = inx.union(cols)

Adf = Adf.reindex(index=inx_un).reindex(columns=inx_un)
Adf = Adf.fillna(0)
Adf = pd.DataFrame(data=(np.maximum(Adf.values, Adf.values.T)),
                   index= Adf.index, columns= Adf.columns)

# AdjM building

In [ ]:
def make_match_node_AdjDfs(df1, df2):
    
    # assumes dfs are symmetric and square
    # should implement a check here...
    
    # get union of dataframe indexes
    inx_1 = df1.index
    inx_2 = df2.index
    inx_un = inx_1.union(inx_2)
    
    # reindex both index and columns filling with 0
    df1 = df1.reindex(index=inx_un, fill_value=0).reindex(columns=inx_un, fill_value=0)
    df2 = df2.reindex(index=inx_un, fill_value=0).reindex(columns=inx_un, fill_value=0)
    
    return df1, df2

In [ ]:
def make_match_node_intersection_AdjDfs(df1, df2):
    
    # assumes dfs are symmetric and square
    # should implement a check here...
    
    # get union of dataframe indexes
    inx_1 = df1.index
    inx_2 = df2.index
    inx_un = inx_1.intersection(inx_2)
    
    # reindex both index and columns for just 
    df1 = df1.reindex(index=inx_un).reindex(columns=inx_un)
    df2 = df2.reindex(index=inx_un).reindex(columns=inx_un)
    
    return df1, df2

## Speed Testing

# DeltaCon

In [81]:
def compute_FBP_invs(A):

    # max power to calculate inverse to
    max_pw = 7

    # make identity
    I = sp.identity(A.shape[0])
 
    # remove self links - can't propogate beliefs to self - need to consider if the self links should be counted in
    # degree still?
    A = A - sp.dia_matrix((A.diagonal(), [0]),
                                    shape=(A.shape[0], A.shape[0]))

    # make degree matrix
    ds = [int(x) for x in A.sum(axis=1)]
    D = sp.dia_matrix((ds, [0]),
                     shape=(len(ds), len(ds)))
    
    # compute about-half homophility factor that gaurentees convergences of W power series
    c1 = D.sum() + 2
    c2 = np.dot(D,D).sum() - 1 
    h_h = sqrt((-c1+sqrt(c1**2 + 4*c2))
               /(8*c2)
              )
    
    # compute system constants ah (from ahD) and ch from (achA)
    ah = 4*(h_h**2) /(1-4*(h_h**2))
    ch = 2*h_h / (1-4*(h_h**2))
    
    # create matrix M to form (I - M)^-1
    M = ch*A - ah*D
    
    # expand to power series to compute inverse
    inv = I
    mat = M
    pwr = 1
    while mat.data[mat.data.argmax()] > 10**(-9) and pwr < max_pw:
        inv = inv + mat
        mat = mat * M
        pwr = pwr + 1
    return inv

In [82]:
def DeltaCon(A1, A2):
    
    # should implement a quick for identical networks once checked 
    
    # initialize
    # prior belief for nodes (doesn't really affect results of deltacon - how not???)
    p = 0.51
    
    # Solve the linear BP systems (FaBP): inv corresponds to [I + ah*D - ch*A]^-1
    # and multiple by about-half prior beliefs
    inv1 = compute_FBP_invs(A1) * (p-0.5)
    inv2 = compute_FBP_invs(A2) * (p-0.5)
    
    # compute the similarities using Matusita distance
    diffM = (inv1.sqrt() - inv2.sqrt())
    diffM = diffM.multiply(diffM)
    
    # delete diagonal entries -  the influence nodes have on themselves
    diffM = diffM - sp.dia_matrix((diffM.diagonal(), [0]), shape=(diffM.shape[0], diffM.shape[0]))

    return 1 / (1 + sqrt(diffM.sum()))

# Applying

In [83]:
def compute_DeltaCon_sims(df, year):
    
    # create dataframe to store similarities
    df_temp = pd.DataFrame()
    
    # for countries in dataset
    for col1 in df:
        print '\n%s' % col1 # to see progress
        
        # if the country1 has no patents then set similarity to -1 to denote this
        if not df.loc[year,col1]:
            for col2 in df:
                df_temp.loc[col1,col2] = -1
            
        # else create adjacency dataframe df1 (via nx.graph) for country1
        else:
            g1 = nx.MultiGraph()
            for ls in df.loc[year,col1]:
                g1.add_edges_from(combinations(ls, 2))
            Adf1 = nx.to_pandas_dataframe(g1)

            # again for countries in dataset 
            for col2 in df:
                
                # if the country2 has no patents then set similarity to -1 to denote this 
                if not df.loc[year,col2]:
                    df_temp.loc[col1,col2] = -1
                    
                # else calculate DeltaCon sim for country1 and country2
                else:
                    print col2, # to see progress
                    
                    # create adjacency dataframe df2 (via nx.graph) for country2
                    g2 = nx.MultiGraph()
                    for ls in df.loc[year,col2]:
                        g2.add_edges_from(combinations(ls, 2))    
                    Adf2 = nx.to_pandas_dataframe(g2)
                    
                    # make adjacency dataframes compatible
                    Adf1, Adf2 = make_match_node_AdjDfs(Adf1, Adf2)
                    
                    # convert to csr and compute DeltaCon
                    df_temp.loc[col1,col2] = DeltaCon(sp.csr_matrix(Adf1.values), sp.csr_matrix(Adf2.values))
                    
    return df_temp